### Perguntas
- Pergunta principal: Qual é a diferença de uso das bicicletas pelos **clientes casuais e anuais (membros)**?
- Perguntas específicas:
> - Qual é a média de duração da locação das bicicletas?
> - Quais são os dias da semana com a maior/menor quantidade de locações?
> - Qual é o total de clientes por dia, mês e ano?
> - Quais são os tipos de bicicleta mais alugados?
> - Quais são as estações mais utilizadas?
> - Qual é a localização geográfica das retiradas e devoluções?
> - Quais são os horários do dia com maiores locações e devoluções?

### Upload dos dados (.csv)
- Todos os arquivos (datasets) foram carregados para um volume dentro do banco de dados (portfolio_citybikes)
- Posteriormente, foram criadas tabelas em lotes de 10 em 10 de maneira manual através da interface do Databricks (/Volumes/)
- A tabela com o modelo de esquema foi criada inicialmente a partir do arquivo "divvy_trips_2020_q_1.csv"

In [0]:
-- Criação da tabela modelo com dados iniciais e alteração de alguns tipos de dados
-- Esta tabela será a base para a tabela fonte com todos os dados consolidados (jan2020 até jul2025)
CREATE TABLE workspace.portfolio_citybikes.tabela_fonte_consolidada AS
SELECT
  ride_id,
  rideable_type,
  started_at,
  ended_at,
  start_station_name,
  CAST(start_station_id AS STRING),
  end_station_name,
  CAST(end_station_id AS STRING),
  start_lat,
  start_lng,
  end_lat,
  end_lng,
  member_casual,
  _rescued_data
  FROM workspace.portfolio_citybikes.divvy_trips_2020_q_1

### Adição de dados na tabela modelo

In [0]:
-- Inclusão de dados na tabela fonte (abr2020 até jan2021)
INSERT INTO workspace.portfolio_citybikes.tabela_fonte_consolidada
TABLE workspace.portfolio_citybikes.202004_202101

In [0]:
-- Inclusão de dados na tabela fonte (fev2021 até nov2021)
INSERT INTO workspace.portfolio_citybikes.tabela_fonte_consolidada
TABLE workspace.portfolio_citybikes.202102_202111

In [0]:
-- Inclusão de dados na tabela fonte (dez2021 até set2022)
INSERT INTO workspace.portfolio_citybikes.tabela_fonte_consolidada
TABLE workspace.portfolio_citybikes.202112_202209

In [0]:
-- Inclusão de dados na tabela fonte (out2022 até jul2023)
INSERT INTO workspace.portfolio_citybikes.tabela_fonte_consolidada
TABLE workspace.portfolio_citybikes.202210_202307

In [0]:
-- Inclusão de dados na tabela fonte (ago2023 até mai2024)
INSERT INTO workspace.portfolio_citybikes.tabela_fonte_consolidada
TABLE workspace.portfolio_citybikes.202308_202405

In [0]:
-- Inclusão de dados na tabela fonte (jun2024 até mar2025)
INSERT INTO workspace.portfolio_citybikes.tabela_fonte_consolidada
TABLE workspace.portfolio_citybikes.202406_202503

In [0]:
-- Inclusão de dados na tabela fonte (abr2025 até jul2025)
INSERT INTO workspace.portfolio_citybikes.tabela_fonte_consolidada
TABLE workspace.portfolio_citybikes.202504_202507

### Verificação de linhas
Verificação da quantidade original de linhas e posteriormente à execução do comando INSERT INTO para a tabela_fonte_consolidada

In [0]:
-- Garantindo que não existe nenhum valor nulo na coluna ride_id
SELECT ride_id
FROM workspace.portfolio_citybikes.tabela_fonte_consolidada
WHERE ride_id IS NULL

In [0]:
-- Contagem do total de linhas da tabela fonte consolidada (verificação)
SELECT COUNT(ride_id)
FROM workspace.portfolio_citybikes.tabela_fonte_consolidada

In [0]:
-- Contagem total utilizando CTE para verificação
WITH contagem_linhas AS (
  SELECT ride_id
  FROM workspace.portfolio_citybikes.divvy_trips_2020_q_1
  UNION ALL
  SELECT ride_id
  FROM workspace.portfolio_citybikes.202004_202101
  UNION ALL
  SELECT ride_id
  FROM workspace.portfolio_citybikes.202102_202111
  UNION ALL
  SELECT ride_id
  FROM workspace.portfolio_citybikes.202112_202209
  UNION ALL
  SELECT ride_id
  FROM workspace.portfolio_citybikes.202210_202307
  UNION ALL
  SELECT ride_id
  FROM workspace.portfolio_citybikes.202308_202405
  UNION ALL
  SELECT ride_id
  FROM workspace.portfolio_citybikes.202406_202503
  UNION ALL
  SELECT ride_id
  FROM workspace.portfolio_citybikes.202504_202507
)
SELECT COUNT(ride_id)
FROM contagem_linhas

### Modelagem SQL
- fact_rides
> - ride_id
> - start_lat
> - start_lng
> - end_lat
> - end_lng
> - started_at
> - ended_at
> - start_station_fk
> - end_station_fk
> - duracao_locacao: coluna calculada (started_at - ended_at)
> - dia_semana_retirada: coluna calculada (WEEKDAY)
> - dia_semana_devolucao: coluna calculada (WEEKDAY)
> - hora_retirada: coluna calculada (HOUR)
> - hora_devolucao: coluna calculada (HOUR)
> - rideable_type_fk
> - member_casual_fk
- dim_rideable_type
- dim_start_station
- dim_end_station
- dim_member_casual

In [0]:
-- Criação de tabela de dimensão (tipo de bicicleta)
CREATE OR REPLACE TABLE workspace.portfolio_citybikes.dim_rideable_type AS
SELECT
  ROW_NUMBER() OVER (ORDER BY rideable_type) AS id_rideable_type,
  rideable_type
FROM
  (SELECT DISTINCT rideable_type FROM workspace.portfolio_citybikes.tabela_fonte_consolidada)

In [0]:
-- Verificação da tabela criada
SELECT *
FROM workspace.portfolio_citybikes.dim_rideable_type

In [0]:
-- Verificação do data_type da tabela criada
DESCRIBE TABLE workspace.portfolio_citybikes.dim_rideable_type

In [0]:
-- Criação de tabela de dimensão (estação inicial)
CREATE OR REPLACE TABLE workspace.portfolio_citybikes.dim_start_station AS
SELECT
  ROW_NUMBER() OVER (ORDER BY start_station_name) AS start_station_id,
  start_station_name
FROM (SELECT DISTINCT start_station_name FROM workspace.portfolio_citybikes.tabela_fonte_consolidada)
WHERE
  start_station_name IS NOT NULL

In [0]:
-- Verificação da tabela criada
SELECT *
FROM workspace.portfolio_citybikes.dim_start_station

In [0]:
-- Verificação do data_type da tabela criada
DESCRIBE TABLE workspace.portfolio_citybikes.dim_start_station

In [0]:
-- Criação de tabela de dimensão (estação final)
CREATE OR REPLACE TABLE workspace.portfolio_citybikes.dim_end_station AS
SELECT
  ROW_NUMBER() OVER (ORDER BY end_station_name) AS end_station_id,
  end_station_name
FROM (SELECT DISTINCT end_station_name FROM workspace.portfolio_citybikes.tabela_fonte_consolidada)
WHERE
  end_station_name IS NOT NULL

In [0]:
-- Verificação da tabela criada
SELECT *
FROM workspace.portfolio_citybikes.dim_end_station

In [0]:
-- Verificação do data_type da tabela criada
DESCRIBE TABLE workspace.portfolio_citybikes.dim_end_station

In [0]:
-- Criação de tabela de dimensão (tipo de cliente)
CREATE OR REPLACE TABLE workspace.portfolio_citybikes.dim_member_casual AS
SELECT
  ROW_NUMBER() OVER (ORDER BY member_casual) AS id_member_casual,
  member_casual
FROM
  (SELECT DISTINCT member_casual FROM workspace.portfolio_citybikes.tabela_fonte_consolidada)

In [0]:
-- Verificação da tabela criada (dim_member_casual)
SELECT *
FROM workspace.portfolio_citybikes.dim_member_casual

In [0]:
-- Criação da fact table
  CREATE OR REPLACE TABLE workspace.portfolio_citybikes.fact_rides AS
  SELECT
    l.ride_id,
    l.start_lat,
    l.start_lng,
    l.end_lat,
    l.end_lng,
    l.started_at,
    l.ended_at,
    -- Criação da coluna de duração da locação (em segundos)
      TIMESTAMPDIFF(SECOND, l.started_at, l.ended_at) AS duracao_locacao,
    -- Criação da coluna de dia da semana de retirada
      WEEKDAY(l.started_at) AS dia_semana_retirada,
    -- Criação da coluna de dia da semana de devolução
      WEEKDAY(l.ended_at) AS dia_semana_devolucao,
    -- Criação da coluna de hora de retirada
      HOUR(l.started_at) AS hora_retirada,
    -- Criação da coluna de hora de devolução
      HOUR(l.ended_at) AS hora_devolucao,  
    r1.id_rideable_type AS rideable_type_fk,
    r2.id_member_casual AS member_casual_fk,
    r3.start_station_id AS start_station_fk,
    r4.end_station_id AS end_station_fk
  FROM workspace.portfolio_citybikes.tabela_fonte_consolidada AS l
  LEFT JOIN workspace.portfolio_citybikes.dim_rideable_type AS r1
    ON l.rideable_type = r1.rideable_type
  LEFT JOIN workspace.portfolio_citybikes.dim_member_casual AS r2
    ON l.member_casual = r2.member_casual
  LEFT JOIN workspace.portfolio_citybikes.dim_start_station AS r3
    ON l.start_station_name = r3.start_station_name
  LEFT JOIN workspace.portfolio_citybikes.dim_end_station AS r4
    ON l.end_station_name = r4.end_station_name
  -- Excluindo valores negativos da duração da locação, em ocorrências onde ended_at é menor que started_at
    WHERE TIMESTAMPDIFF(SECOND, l.started_at, l.ended_at) > 0

In [0]:
-- Verificação da tabela criada
SELECT *
FROM workspace.portfolio_citybikes.fact_rides

In [0]:
-- Alterando comentário da coluna para informar que a duração da locação está em segundos
ALTER TABLE workspace.portfolio_citybikes.fact_rides
ALTER COLUMN duracao_locacao COMMENT 'Duração da locação em segundos'

In [0]:
-- Verificação do data_type da tabela criada
DESCRIBE TABLE workspace.portfolio_citybikes.fact_rides